In [16]:
import os
import itertools as itt
import shutil
import nibabel as nib
import nibabel.freesurfer.io as fs
import numpy as np
import pandas as pd
from IPython import embed
import subprocess
import sys
import shutil


Modified orginal csv renamed to MRS_data_curated_+paths+avg_skeleton+registration.csv
- correct the filepath for T1 and DWI
- Save the filepath for outputs

Since later we decided to do analysis in T1 space, I updated the csv with new paths for volumes to be saved in T1 space and named it as completed.csv

In [17]:
print(os.environ['PATH'])
os.environ['PATH'] += os.pathsep + '/home/localadmin/ANTS_bin/install/bin/'
csv_file_path = '/home/localadmin/hpc_mount/wangqiaochu/completed.csv'
df = pd.read_csv(csv_file_path)

/bin:/home/localadmin/.local/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/local/go/bin:/usr/local/go/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/fsl/share/fsl/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/local/go/bin:/usr/local/go/bin:/home/localadmin/ANTS_bin/install/bin/:/home/localadmin/ANTS_bin/install/bin/:/home/localadmin/ANTS_bin/install/bin/


Use antsRegistration to register DWI to T1, Here I preserved the flag settings in white matter analysis.

In [18]:
def quickWarp(f, m, o, debug=False, metric='MI', conv=6, itsc=1, rdsc=1, smosc=1):
    """antsQuickReg: f=fixed, m=moving, o=outPrefix, conv=convergence thr, itsc=base number iterations * scale"""
    if metric == 'CC':
        rb = 1 * int(rdsc)  # radius
    else:
        rb = 32  # bins
    itsc = int(itsc)
    l0, l1, l2, l3, rep = 1000 * itsc, 500 * itsc, 250 * itsc, 100 * itsc, 10 * int(1 + itsc / 2)
    s0, s1, s2, s3, = int(3 * smosc), int(2 * smosc), int(1 * smosc), int(0 * smosc)
    
    call = [
        f'antsRegistration ',
        f' --verbose {int(debug)} ', # print time taken for each iteration and debug info
        f' --dimensionality 3 ', # 3D volumes
        f' --collapse-output-transforms 1', 
        #   collapse output transforms. Specifically, enabling this option combines all 
        #   adjacent transforms wherepossible. All adjacent linear transforms are written to 
        #   disk in the forman itk affine transform (called xxxGenericAffine.mat). 
        #   Similarly, all adjacent displacement field transforms are combined when written 
        #   to disk (e.g. xxxWarp.nii.gz and xxxInverseWarp.nii.gz (if available)).Also, an 
        #   output composite transform including the collapsed transforms is written to the 
        #   disk (called outputCollapsed(Inverse)Composite). 
        f' --float 0 ', # use float instead of double
        f' --output [ {o}, {o}Warped.nii.gz,{o}InverseWarped.nii.gz ] ',
        #   [outputTransformPrefix,<outputWarpedImage>,<outputInverseWarpedImage>]
        #   Specify the output transform prefix (output format is .nii.gz )
        f' --interpolation Linear', 
        f' --use-histogram-matching 0', # Not use histogram matching before registration
        f' --winsorize-image-intensities [ 0.005,0.995 ] ', # Winsorization clips outlier intensities during registration. 
        f' --initial-moving-transform [ {f},{m},1 ] ',
        #   An initial moving transform is used to quickly align the images before registration. 
        #   The optimization process requires images to roughly overlap and will fail if images are too far apart in physical space. 
        #   There are various options for the initialization: 0-match by mid-point (i.e., center voxel of one image will be brought in line with center voxel of the other)
        #   1-match by center of mass
        #   2-match by point of origin (i.e. physical coordinates 0,0,0, as defined by the image headers) The center of mass alignment usually works well.
        f' --transform Rigid[ 0.1 ] ', # gradient around 0.1-0.25
        f' --metric {metric}[ {f},{m},1,{rb},Regular, 0.25 ] ',
        #   All ANTs metrics have the form [fixed, moving, parameters],
        #   Here paramters are [weight, bins, sampling method, sampling rate]
        f' --convergence [ {l0}x{l1}x{l2}x{l3},1e-{conv},{rep} ] ',
        #   we will run 4 levels (multi-resolution steps) with a maximum number of iterations of
        #   1000,500,250,100. The threshold (1e-6) tells the algorithm to stop if the improvement 
        #   in mutual information has not changed more than 1e-6 in the last 10 iterations 
        #   (convergenceWindowSize=10).
        f' --shrink-factors 8x4x2x1',
        #   the 4 hierarchical steps will have resolutions divided by 8,4,2,1. The number of "shrink factors" must 
        #   match the number of levels in the --convergence option. The factors use the fixed image as reference. 
        #   If the fixed image has spacing 1x1x1mm, and the moving image has spacing 2x2x2mm, then --shrink-factors 
        #   4x2x1 will register images at 4mm resolution, then 2mm, then 1mm. But if you switch the fixed and moving images,
        #   --shrink-factors 4x2x1 will register images at 8mm, then 4mm, then 2mm. For 3D images, 
        #   shrinking the volumes by a factor of 2 decreases the number of voxels by a factor of 8. 
        #   So running iterations at shrink factor 1 requires about 8 times as much computation as at shrink factor 2, and so on.
        #   f' --smoothing-sigmas 3x2x1x0vox ',
        #   The smoothing is Gaussian with a kernel standard deviation of 3,2,1,0 voxels. Here they are specified in voxels,
        #   but they can also be in mm. To convert the sigma to FWHM you can use roughly a factor of 2.36. The images are smoothed
        #   before being downsampled at each level. The number of smoothing levels must match the number of iteration levels --convergence
        f' --transform Affine[ 0.1 ] ', # traslation, rotation, shearing, zooming
        f' --metric {metric}[ {f},{m},1,{rb},Regular, 0.25 ] ',
        f' --convergence [ {l0}x{l1}x{l2}x{l3},1e-{conv},{rep} ] ',
        f' --shrink-factors 8x4x2x1',
        f' --smoothing-sigmas 3x2x1x0vox',
        f' --transform SyN[ 0.2, 3 ,0 ]', # nonlinear [gradientStep,updateFieldVarianceInVoxelSpace,totalFieldVarianceInVoxelSpace]
        f' --metric {metric}[ {f},{m},1,{rb}, Regular,0.25 ]',
        f' --convergence [{l0}x{l1}x{l2}x{l3},1e-{conv + 1},{rep}]',
        f' --shrink-factors 8x4x2x1',
        f' --smoothing-sigmas {s0}x{s1}x{s2}x{s3}vox'
    ]
    ##https://github.com/ANTsX/ANTs/wiki/Anatomy-of-an-antsRegistration-call
 
    if debug:
        print(call)
    
    return os.system(' '.join(call))


Upsample the DWI to match T1 resolution
- T1: 1.2, 1.2, 1
- B0: 2.208, 2.208, 3

In [19]:
def _ANTs_ResampleImage(inimg, outimg='path/name', spacing=None, size_or_spacing=0, interp=0, dim=3):
    """Resample an image based on voxel spacing.
    
    Parameters:
    - inimg: Input image (e.g., DWI image)
    - outimg: Output image path
    - spacing: List of 3 values representing the desired voxel spacing (e.g., [1.2, 1, 1] for T1 resolution)
    - size_or_spacing: Use 0 for voxel spacing (default), 1 for size (number of voxels)
    - interp: Interpolation type (0=nearest neighbor, 1=linear, etc.)
    - dim: Image dimensionality (3 for 3D)
    """
    
    if spacing is None:
        raise ValueError('ERROR: spacing must be a 3D list, e.g.: [1.2, 1, 1]')
    
    # Convert the spacing list to a string formatted as 'MxNxO' (e.g., '1.2x1x1')
    spacing_ = 'x'.join([str(s) for s in spacing])
    
    # Execute the ResampleImage command with voxel spacing
    os.system(f'ResampleImage {dim} {inimg} {outimg} {spacing_} {size_or_spacing} {interp} 6')
    
    return outimg

Add cras to default freesurfer space, which results in T1 space.

In [ ]:
def add_FScras_to_surfaces(surface, output_dir, talairachlta='/mri/transforms/talairach.lta'):
    """
    Apply the CRAS offset from the Talairach file to the FreeSurfer surface coordinates.
    Save the shifted surface to a new file in the specified output directory.
    """
    # Load the FreeSurfer surface file (e.g., lh.white)
    coords, faces = fs.read_geometry(surface)
    
    # Load CRAS offset from the Talairach file
    with open(talairachlta, 'r') as fp:
        cras = [np.fromstring(l[9:-1], sep=' ') for l in fp if 'cras' in l][0]
    
    print('Offsetting:', surface)
    print('CRAS: ', cras)
    
    # Apply the CRAS offset to the coordinates
    coords += cras
    
    # Create a new filename for the shifted surface
    surface_name = os.path.basename(surface)
    shifted_surface_name = f"{surface_name}"
    shifted_surface = os.path.join(output_dir, shifted_surface_name)

    
    # Save the modified surface
    fs.write_geometry(shifted_surface, coords, faces)
    
    print(f"Shifted surface saved to: {shifted_surface}")
    
    return shifted_surface

Metrics saved at /home/localadmin/hpc_mount/wangqiaochu/sub-CHUV{sub}/{para}.nii.gz

B0 saved at /home/localadmin/hpc_mount/wangqiaochu/sub-CHUV{sub}/B0_to_T1_sub-CHUV{sub}_Warped.nii.gz

surfaces saved at /home/localadmin/hpc_mount/wangqiaochu/sub-CHUV{sub}

All in T1 space


In [ ]:
# T1 voxel dimensions in mm
spacing = [1.2, 1, 1]  

# Read the subject list from CSV
subject_list = ['sub-CHUV' + sub for sub in df['sub'].tolist()]

# Base paths
base_surf_dir = '/home/localadmin/hpc_mount/VBAmrs/LSP_subset'
base_transform_dir = '/home/localadmin/hpc_mount/wangqiaochu'
base_output_dir = '/home/localadmin/hpc_mount/wangqiaochu'

# Surface types to process
surfaces = ['lh.white', 'lh.pial', 'rh.white', 'rh.pial']

# Register B0 to T1
for index, row in df.iterrows(): 
        t1_path = row['t1_path']
        dwi_path = row['dwi_path']
        output_prefix = row['output_prefix']
        resampled_dwi_output = row['resampled_dwi_output']
        registered_dwi = row['registered_dwi']
        sub = row['sub']

        # Resample DWI to match T1 resolution
        _ANTs_ResampleImage(inimg=dwi_path, outimg=resampled_dwi_output, spacing=spacing, interp=1, dim=3)

        # Perform registration on upsampled DWI
        quickWarp(f=t1_path, m=resampled_dwi_output, o=output_prefix, debug=False, metric='MI', conv=6, itsc=1, rdsc=1, smosc=1)

        # Apply the transformation to the original DWI
        subprocess.run([
            'antsApplyTransforms',
            '--dimensionality', '3',
            '--input', dwi_path,
            '--reference-image', t1_path,
            '--output', registered_dwi,
            '--transform', f'{output_prefix}1Warp.nii.gz',
            '--transform', f'{output_prefix}0GenericAffine.mat',
            '--interpolation', 'Linear'
        ])
        print('finished', sub)
   

# Register metrics and surfaces to T1
for index, row in df.iterrows():
    subject_id = 'sub-CHUV' + row['sub']
    session = 'ses-' + row['ses']
    t1_path = row['t1_path']
    output_prefix = row['output_prefix']
    
    # Paths for surface and Talairach files
    surf_dir = f'{base_surf_dir}/{subject_id}/{session}/fastsurfer/surf/'
    talairach_lta = f'{base_surf_dir}/{subject_id}/{session}/fastsurfer/mri/transforms/talairach.lta'

    # Transformation files (specific to each subject)
    transform_dir = f'{base_transform_dir}/{subject_id}'
    transforms = [
        f'{transform_dir}/B0_to_T1_{subject_id}_1Warp.nii.gz',
        f'{transform_dir}/B0_to_T1_{subject_id}_0GenericAffine.mat'
    ]

    # Output directory
    output_dir = f'{base_output_dir}/{subject_id}'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for para in ['ad', 'md', 'rd', 'fa', 'ak', 'mk', 'rk']:
        dwifit_path = row[f'dwifit_path_{para}']
        dwifit_inT1 = row[f'dwifit_inT1_{para}']
        
        # Construct the command
        command = [
            'antsApplyTransforms',
            '--dimensionality', '3',
            '--input', dwifit_path,
            '--reference-image', t1_path,mki.gz',
            '--transform', f'{output_prefix}0GenericAffine.mat',
            '--interpolation', 'Linear'
        ]
        
        # Run the command
        print(f"Running for subject {subject_id}, parameter {para}...")
        subprocess.run(command, check=True)
        print(f"Finished {subject_id}, parameter {para}")
        
    for surface in surfaces:
        surface_path = os.path.join(surf_dir, surface)
        
        # Step 1: Shift surface from FSA space to original RAS space
        shifted_surface = add_FScras_to_surfaces(surface_path, output_dir, talairachlta=talairach_lta)

        print(f"Processed surface {surface} for {subject_id}, saved at: {shifted_surface}")
        